Sunay Sanghani
USC ID: 5373536322
Github Username: sunaysanghani

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv
import numpy as np
from sklearn.utils import resample

In [16]:
folder_path = "C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data"
def split_train_test(root):
    train = []
    test = []
    for root, directory, file in os.walk(root):
        for f in file:
            if 'pdf' not in f:
                new_name = os.path.join(root, f)
                if 'bending1' in new_name or 'bending2' in new_name:
                    if 'dataset1.csv' in new_name or 'dataset2.csv' in new_name:
                        test.append(new_name)
                    else:
                        train.append(new_name)
                else:
                    if 'dataset1.csv' in new_name or 'dataset2.csv' in new_name or 'dataset3.csv' in new_name:
                        test.append(new_name)
                    else:
                        train.append(new_name)
    return train, test

In [17]:
train, test = split_train_test(folder_path)

In [18]:
train

['C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset3.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset4.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset5.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset6.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset7.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset3.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset4.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset5.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset6.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset10.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset11.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset12.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data

In [19]:
test

['C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset1.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending1\\dataset2.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset1.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\bending2\\dataset2.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset1.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset2.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\cycling\\dataset3.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\lying\\dataset1.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\lying\\dataset2.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\lying\\dataset3.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\sitting\\dataset1.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\sitting\\dataset2.csv',
 'C:/Users/sunay/Downloads/DSCI552HW3/data/Arem_data\\sitting\\dat

# 1c. Feature Extraction

Research what types of time-domain features are usually used in time series
classification and list them (examples are minimum, maximum, mean, etc).

1. Date time features
2. mean
3. minimum
4. standard deviation
5. first quartile
6. third quartile
7. distribution
8. correlation graphs

# 1(c)(ii). Extract

In [20]:
columns_to_measure = ['min', 'max', 'mean', 'median', 'std', 'first_q', 'third_q']
columns_df = [col + str(i) for i in range(1,7) for col in columns_to_measure]
sniffer = csv.Sniffer()

params = ['min', 'max', 'mean', 'std', '25%', '50%', '75%']
extract_data = []
def get_path_for_file(file):
    directory = os.path.dirname(file)
    components = os.path.dirname(file).split("/")
    return components[-1]

def _separator(file):
    f = open(file)
    return sniffer.sniff(f.read()).delimiter

def load_Data(files, features, cols, segs=1, standard=False):
    instances = []
    labels = []
    for f in files:
        labels.append(get_path_for_file(f))
        separator = _separator(f)
        if separator == ",":
            new_file = pd.read_csv(f, skiprows=5, header=None, on_bad_lines='skip')
        else:
            new_file = pd.read_csv(f, skiprows=5,sep="\\s+", header=None, on_bad_lines='skip')
        new_file.columns = ["# Columns: time","avg_rss12","var_rss12","avg_rss13","var_rss13","avg_rss23","var_rss23"]

        
        segments = int(new_file.shape[0] / segs)
        description = []
        for i in range(segs):
            start_length = i * segments
            end_length= min((i+1) * segments, new_file.shape[0])
            current_segment = new_file[start_length : end_length]
            time_column = "# Columns: time"
            current_description = current_segment.describe().drop('count').drop(columns=time_column).T
            description.append(current_description.values.flatten())
        instances.append(np.concatenate(description))
    feature_df = pd.DataFrame(instances)
    feature_df.columns = columns_df
    feature_df = feature_df.loc[:, features]
    
    feature_df['label'] = pd.Series(labels)
    
    return feature_df

In [21]:
features = [f'{stat}{i}' for i in range(1,7) for stat in columns_to_measure]

In [22]:
train_data = load_Data(train, features, columns_to_measure)
test_data = load_Data(test, features, columns_to_measure)

In [23]:
train_data

,min1,max1,mean1,median1,std1,first_q1,third_q1,min2,max2,mean2,...,first_q5,third_q5,min6,max6,mean6,median6,std6,first_q6,third_q6,label
0,43.954500,1.558835,35.00,43.00,44.330,45.0000,47.40,0.426250,0.338690,0.0,...,36.50,38.50,0.493292,0.513506,0.0,0.00,0.430,0.9400,1.79,Arem_data\bending1
1,42.179813,3.670666,33.00,39.15,43.500,45.0000,47.75,0.696042,0.630860,0.0,...,36.33,38.67,0.613521,0.524317,0.0,0.00,0.500,1.0000,2.18,Arem_data\bending1
2,41.678063,2.243490,33.00,41.33,41.750,42.7500,45.75,0.535979,0.405469,0.0,...,31.25,37.50,0.383292,0.389164,0.0,0.00,0.430,0.5000,1.79,Arem_data\bending1
3,43.454958,1.386098,37.00,42.50,43.250,45.0000,48.00,0.378083,0.315566,0.0,...,24.00,33.50,0.679646,0.622534,0.0,0.43,0.500,0.8700,5.26,Arem_data\bending1
4,43.969125,1.618364,36.25,43.31,44.500,44.6700,48.00,0.413125,0.263111,0.0,...,23.75,30.75,0.555313,0.487826,0.0,0.00,0.490,0.8300,2.96,Arem_data\bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,34.763333,4.742208,20.75,31.67,35.290,38.2500,46.25,4.223792,2.278004,0.0,...,18.33,25.50,3.288271,1.647528,0.0,2.05,3.270,4.3050,9.39,Arem_data\walking
65,34.935813,4.645944,21.50,32.00,35.500,38.0625,51.00,4.115750,2.335243,0.0,...,18.25,25.00,3.280021,1.700918,0.0,2.12,3.015,4.5000,10.21,Arem_data\walking
66,34.333042,4.948770,18.33,31.25,34.750,38.0000,47.67,4.396958,2.568680,0.0,...,18.00,24.00,3.261583,1.617290,0.0,2.05,2.980,4.3200,8.01,Arem_data\walking
67,34.599875,4.731790,18.33,31.50,35.125,38.0000,45.75,4.398833,2.538521,0.0,...,18.25,23.25,3.289542,1.680170,0.0,2.12,3.015,4.2600,8.86,Arem_data\walking


In [24]:
test_data

,min1,max1,mean1,median1,std1,first_q1,third_q1,min2,max2,mean2,...,first_q5,third_q5,min6,max6,mean6,median6,std6,first_q6,third_q6,label
0,40.624792,1.476967,37.25,39.2500,40.500,42.0000,45.00,0.358604,0.322605,0.0,...,36.0000,38.25,0.570583,0.582915,0.0,0.0000,0.430,1.300,1.92,Arem_data\bending1
1,42.812812,1.435550,38.00,42.0000,42.500,43.6700,45.67,0.372438,0.289158,0.0,...,34.5000,38.50,0.571083,0.601010,0.0,0.0000,0.430,1.300,3.11,Arem_data\bending1
2,24.562958,3.737514,12.75,23.1875,24.250,26.5000,51.00,0.590833,0.837408,0.0,...,27.0000,30.00,0.700188,0.693720,0.0,0.4300,0.500,0.870,4.97,Arem_data\bending2
3,27.464604,3.583582,0.00,25.5000,28.000,30.0000,42.75,0.449708,0.767197,0.0,...,20.7500,33.00,1.122125,1.012342,0.0,0.4700,0.830,1.300,6.76,Arem_data\bending2
4,37.177042,3.581301,24.25,34.5000,36.250,40.2500,45.00,2.374208,1.601799,0.0,...,21.7500,25.50,2.921729,1.852600,0.0,1.5000,2.500,3.900,9.34,Arem_data\cycling
5,37.561188,3.226507,28.75,35.2500,36.875,40.2500,44.75,2.080688,1.639258,0.0,...,21.5000,24.33,2.765896,1.769203,0.0,1.4100,2.450,3.770,9.62,Arem_data\cycling
6,37.058708,3.710180,22.00,34.5000,36.000,40.0625,44.67,2.438146,1.996887,0.0,...,21.0000,24.25,2.983750,1.815730,0.0,1.5000,2.570,4.150,8.55,Arem_data\cycling
7,27.716375,1.442253,23.50,27.0000,27.500,29.0000,30.00,0.363688,0.394817,0.0,...,10.7500,21.00,0.734271,0.613688,0.0,0.4300,0.710,1.000,4.50,Arem_data\lying
8,44.182937,7.495615,24.75,48.0000,48.000,48.0000,48.33,0.101875,0.346756,0.0,...,5.5425,12.75,0.692771,0.675781,0.0,0.3225,0.500,0.940,3.91,Arem_data\lying
9,48.004167,0.032038,48.00,48.0000,48.000,48.0000,48.25,0.007167,0.055106,0.0,...,10.0000,12.00,0.641229,0.388372,0.0,0.4600,0.500,0.830,2.50,Arem_data\lying


In [25]:
train_data.describe().loc['std']

min1        4.917692
max1        1.758670
mean1       8.794295
median1     5.731647
std1        4.956111
first_q1    4.783645
third_q1    4.429182
min2        1.600701
max2        0.902808
mean2       0.000000
median2     0.952201
std2        1.436960
first_q2    2.158416
third_q2    5.147841
min3        3.863097
max3        0.995959
mean3       3.053869
median3     4.145255
std3        3.845730
first_q3    3.946023
third_q3    4.759853
min4        1.179861
max4        0.473576
mean4       0.000000
median4     0.842501
std4        1.150092
first_q4    1.566564
third_q4    2.302408
min5        5.120426
max5        1.057998
mean5       5.368786
median5     5.543882
std5        5.267414
first_q5    4.957231
third_q5    5.449726
min6        1.171401
max6        0.519420
mean6       0.051766
median6     0.774358
std6        1.104626
first_q6    1.550393
third_q6    2.540166
Name: std, dtype: float64

In [26]:
test_data.describe().loc['std']

min1         6.790086
max1         1.869285
mean1       12.136206
median1      7.673052
std1         7.088085
first_q1     6.385720
third_q1     4.379342
min2         1.500529
max2         0.832472
mean2        0.000000
median2      0.941307
std2         1.345903
first_q2     2.028482
third_q2     4.870395
min3         4.588252
max3         0.760779
mean3        2.644618
median3      4.554121
std3         4.753522
first_q3     5.014339
third_q3     5.393220
min4         1.146338
max4         0.409008
mean4        0.000000
median4      0.867088
std4         1.161705
first_q4     1.541943
third_q4     1.733937
min5         7.366781
max5         0.919274
mean5        8.252947
median5      7.823870
std5         7.438749
first_q5     7.259433
third_q5     6.782153
min6         1.119410
max6         0.525263
mean6        0.000000
median6      0.711956
std6         1.037811
first_q6     1.461470
third_q6     2.505306
Name: std, dtype: float64

In [27]:
sampled_std = pd.DataFrame([resample(train_data.iloc[:, :-1]).std() for _ in range(1000)])

interval = sampled_std.quantile([0.05, 0.95])

interval = interval.round(2)

interval = interval.T.rename(columns={0.05: 'lower bound', 0.95: 'higher bound'})

print(interval)

          lower bound  higher bound
min1             4.32          5.38
max1             1.53          1.93
mean1            7.55          9.95
median1          5.14          6.20
std1             4.31          5.47
first_q1         3.91          5.51
third_q1         3.28          5.39
min2             1.41          1.74
max2             0.81          0.96
mean2            0.00          0.00
median2          0.82          1.05
std2             1.26          1.58
first_q2         1.91          2.34
third_q2         4.62          5.56
min3             3.14          4.42
max3             0.76          1.19
mean3            2.83          3.23
median3          3.43          4.67
std3             3.09          4.41
first_q3         3.22          4.52
third_q3         3.94          5.38
min4             1.08          1.24
max4             0.43          0.50
mean4            0.00          0.00
median4          0.77          0.89
std4             1.05          1.21
first_q4         1.43       

# 1iv. Use your judgement to select the three most important time-domain features

1. Mean is a good indicator for time because it has the ability to get a central value when the data distribution is c
2. Median is another good indicator for the data since if it has too many outliers it can get the central value and not get too skewed away from the mean. Median could be a good feature because it does not get as affected by such outliers.
3. Standard Deviation is a good indicator as well for time because it can help us understand the distribution of the data and identify how far away the distribution is away from the mean.

# 2. ISLR 3.7.4

# (a)

Overarching concept is that the smaller the RSS, the better the fit of the model. The cubic model which has more predictors would have a lower RSS compared to a linear regression. Cubic regression has more predictors and has a better fit on the model which would make its RSS better and lower.

# (b)

The testing RSS for cubic regression would be higher than a regular linear regression. The cubic regression has more predictors which leads to overfitting. In the test case, linear regression has a better fit on the data and it's RSS is lower.

# c)

We would expect the RSS for the cubic regression to be less since it still has more predictors. The linear regression will now start to underfit the data, while the cubic regression has more flexibility.

# d)

Unable to answer this since the answer depends on which regression is closer to the actual answer.